In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/boaviztaenvironmental-footprint-data/boavizta-data-us.csv


In [4]:
import pandas as pd

# Load dataset
df = pd.read_csv("/kaggle/input/boaviztaenvironmental-footprint-data/boavizta-data-us.csv")

# Ensure the required columns exist
required_columns = ["manufacturer", "name", "category", "gwp_total", "gwp_use_ratio", 
                    "yearly_tec", "lifetime", "use_location", "gwp_manufacturing_ratio",
                    "weight", "assembly_location", "screen_size", "hard_drive", "memory", 
                    "number_cpu", "server_type", "height"]

# Filter dataset to only include relevant columns
df = df[required_columns]

# Calculate total CO₂ emissions per year based on lifetime
df["gwp_per_year"] = df["gwp_total"] / df["lifetime"]

# Generate LLM input text
df["LLM_Input"] = df.apply(lambda row: f"The {row['manufacturer']} {row['name']} ({row['category']}) has a total estimated carbon footprint of {row['gwp_total']} kgCO₂eq over its {row['lifetime']} years of use. "
                                        f"About {row['gwp_use_ratio']*100:.2f}% of this impact comes from the usage phase, with an estimated yearly energy demand of {row['yearly_tec']} kWh. "
                                        f"The device is used mainly in {row['use_location']} and was assembled in {row['assembly_location']}. "
                                        f"The manufacturing phase contributes {row['gwp_manufacturing_ratio']*100:.2f}% to the total emissions. "
                                        f"The device weighs {row['weight']} kg and has {row.get('memory', 'N/A')}GB RAM, {row.get('hard_drive', 'N/A')} storage, and {row.get('number_cpu', 'N/A')} CPUs. "
                                        f"For servers, the type is {row.get('server_type', 'N/A')}, with a height of {row.get('height', 'N/A')}U.", axis=1)

# Save processed data as CSV
#df[["LLM_Input"]].to_csv("processed_carbon_footprint_llm.csv", index=False)

#print("Processing complete! The LLM-ready dataset is saved as 'processed_carbon_footprint_llm.csv'.")


In [5]:
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,manufacturer,name,category,gwp_total,gwp_use_ratio,yearly_tec,lifetime,use_location,gwp_manufacturing_ratio,weight,assembly_location,screen_size,hard_drive,memory,number_cpu,server_type,height,gwp_per_year,LLM_Input
0,Apple,13-inch MacBook Air (M1 CPU) 256GB - 2020,Workplace,161.0,0.15,NaN,3.0,WW,0.76,NaN,NaN,13.0,SSD 256GB,NaN,NaN,NaN,NaN,53.666667,The Apple 13-inch MacBook Air (M1 CPU) 256GB -...
1,Apple,13-inch MacBook Air (M1 CPU) 512GB - 2020,Workplace,181.0,0.15,NaN,3.0,WW,0.76,NaN,NaN,13.0,SSD 512GB,NaN,NaN,NaN,NaN,60.333333,The Apple 13-inch MacBook Air (M1 CPU) 512GB -...
2,Apple,13-inch MacBook Air with Retina display (1.1GH...,Workplace,174.0,0.15,NaN,3.0,WW,0.77,NaN,NaN,13.0,SSD 256GB,NaN,NaN,NaN,NaN,58.000000,The Apple 13-inch MacBook Air with Retina disp...
3,Apple,13-inch MacBook Air with Retina display (1.1GH...,Workplace,202.0,0.15,NaN,3.0,WW,0.77,NaN,NaN,13.0,SSD 512GB,NaN,NaN,NaN,NaN,67.333333,The Apple 13-inch MacBook Air with Retina disp...
4,Apple,13-inch MacBook Pro (1.4GHz quad-core CPU) 256...,Workplace,217.0,0.17,NaN,3.0,WW,0.76,NaN,NaN,13.0,SSD 256GB,NaN,NaN,NaN,NaN,72.333333,The Apple 13-inch MacBook Pro (1.4GHz quad-cor...


In [6]:
# Select only the LLM_Input column
df_electronics_llm = df[["LLM_Input"]]

# Save to CSV
df_electronics_llm.to_csv("preprocessed_carbon_data_electronics_llm.csv", index=False)

In [7]:
df_electronics_llm

,LLM_Input
0,The Apple 13-inch MacBook Air (M1 CPU) 256GB -...
1,The Apple 13-inch MacBook Air (M1 CPU) 512GB -...
2,The Apple 13-inch MacBook Air with Retina disp...
3,The Apple 13-inch MacBook Air with Retina disp...
4,The Apple 13-inch MacBook Pro (1.4GHz quad-cor...
...,...
1221,The Seagate Nytro 1551 XA1920ME10083 1.92 TB (...
1222,The Seagate Nytro 3331 XS7680SE70004 7.68TB (D...
1223,The Seagate OneStor SP2584 Extensible Storage ...
1224,The Seagate Pulsar 2 800GB SSD (Datacenter) ha...
